In [51]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import lyricsgenius
import re
import requests
from Levenshtein import ratio
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import sys

Evaluating the results of the scraping from 3 different databases:
 - Genius
 - AZLyrics
 - lyrics.az

In [58]:
BB = pd.read_csv("./data/Hot Stuff.csv")
genius = pickle.load(open("./fat/GeniusGithub_reformat.pickle", "rb"))
lyricsaz = pickle.load(open("za_lyr.pickle","rb"))
azlyrics = pickle.load(open("az_lyr.pickle","rb"))

In [59]:
song_art = set()
for i,r in BB.iterrows():
    song_art.add((r['Song'].lower(),r['Performer'].lower()))

In [60]:
print("Total songs: {}".format(len(song_art)))
print("azlyrics songs: {}".format(len({(s.lower(),a.lower())for s,a in azlyrics})))
print("lyricsaz songs: {}".format(len({(s.lower(),a.lower())for s,a in lyricsaz if lyricsaz[(s,a)]})))
print("genius songs: {}".format(len({(s.lower(),a.lower()) for a in genius for s in genius[a]})))

Total songs: 27945
azlyrics songs: 16204
lyricsaz songs: 19864
genius songs: 20188


In [55]:
empty_3_list = lambda : [None]*3
found_songs = defaultdict(empty_3_list)
for s,a in lyricsaz:
    if found_songs[(s.lower(),a.lower())][0]:
        print("already set url to: '{}'".format(found_songs[(s.lower(),a.lower())][0]))
        print("setting url to    : '{}'".format(lyricsaz[(s,a)]))
    found_songs[(s.lower(),a.lower())][0] = lyricsaz[(s,a)]
print("Done with loading lyricsaz data")
for s,a in azlyrics:
    if found_songs[(s.lower(),a.lower())][1]:
        print("already set url to: '{}'".format(found_songs[(s.lower(),a.lower())][1]))
        print("setting url to    : '{}'".format(azlyrics[(s,a)][0][2]))
    found_songs[(s.lower(),a.lower())][1] = azlyrics[(s,a)]
print("Done with loading azlyrics data")
for a in genius:
    for s in genius[a]:
        if found_songs[(s.lower(),a.lower())][2]:
            print("already set url to: '{}'".format(found_songs[(s.lower(),a.lower())][2]))
            print("setting url to    : '{}'".format(genius[a][s]["url"]))
        found_songs[(s.lower(),a.lower())][2] = genius[a][s]["url"]
print("Done with loading genius data")

Done with loading lyricsaz data
Done with loading azlyrics data
already set url to: 'https://genius.com/Xxxtentacion-everybody-dies-in-their-nightmares-lyrics'
setting url to    : 'https://genius.com/Xxxtentacion-everybody-dies-in-their-nightmares-lyrics'
already set url to: 'https://genius.com/Xxxtentacion-jocelyn-flores-lyrics'
setting url to    : 'https://genius.com/Xxxtentacion-jocelyn-flores-lyrics'
already set url to: 'https://genius.com/Blackbear-do-re-mi-lyrics'
setting url to    : 'https://genius.com/Blackbear-do-re-mi-lyrics'
Done with loading genius data


In [56]:
print("scraped songs: {}".format(len(found_songs)))
print("{} unique to lyricsaz".format(sum(int(found_songs[k][1] == None and found_songs[k][2] == None) for k in found_songs)))
print("{} unique to azlyrics".format(sum(int(found_songs[k][0] == None and found_songs[k][2] == None) for k in found_songs)))
print("{} unique to genius".format(sum(int(found_songs[k][0] == None and found_songs[k][1] == None) for k in found_songs)))
print("{} found in all 3".format(sum(int(all(found_songs[k])) for k in found_songs)))

scraped songs: 23350
1493 unique to lyricsaz
703 unique to azlyrics
1245 unique to genius
12941 found in all 3


In [104]:
if len(set(found_songs.keys())-song_art) == 0:
    print("We didn't find any unnecesary info")
else:
    print("For some reason these extra songs were incorporated...")
    print(set(found_songs.keys())-song_art)

We didn't find any unnecesary info


In [46]:
print("All in all we recovered about {}% of the popular songs from BB".format(len(found_songs)/len(song_art)))

All in all we recovered about 0.8355698693862945% of the popular songs from BB


In [106]:
print("10 random songs which weren't found:")
for sa in np.random.choice(["{} by {}".format(s,a) for s,a in song_art-set(found_songs.keys())],10,False):
    print(sa)

10 random songs which weren't found:
right away by hawks
ooh baby by deon jackson
the impossible dream (the quest) by jack jones
mexican pearls by billy vaughn and his orchestra
roller coaster by the ides of march
help me find a way (to say i love you) by little anthony and the imperials
all roads (lead back to you) by donny most
please, mr. president by paula webb
it's too late, baby too late by arthur prysock
bust out by the busters


In [107]:
pickle.dump(dict(found_songs),open("found_songs_links.pickle","wb"))

In [6]:
found_songs = pickle.load(open("found_songs_links.pickle","rb"))

In [109]:
len(test)

23350

Finally grab the lryics for the songs from AZLyrics

In [7]:
az_urls = {k:found_songs[k][1] for k in found_songs if found_songs[k][1]}
test_url = list(az_urls.values())[0]
print(test_url)
print(len(az_urls))

https://www.azlyrics.com/lyrics/kiss/christinesixteen.html
16204


In [29]:
proxies = {
  'http': 'http://10.10.1.10:3128',
  'https': 'http://104.248.108.33:8080',
}
resp = requests.get(test_url,proxies=proxies)
#resp = requests.request("GET",test_url)

ProxyError: HTTPSConnectionPool(host='www.azlyrics.com', port=443): Max retries exceeded with url: /lyrics/kiss/christinesixteen.html (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response',)))

In [123]:
print(resp.text)

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="width=device-width, initial-scale=1">
<meta name="description" content="Lyrics to &quot;Christine Sixteen&quot; song by Kiss: She's got me dizzy, she sees me through to the end She's got me in her hands and there's no use in..."> 
<meta name="keywords" content="Christine Sixteen lyrics, Kiss Christine Sixteen lyrics, Kiss lyrics">
<meta name="robots" content="noarchive">
<meta property="og:image" content="//www.azlyrics.com/az_logo_tr.png">
<title>Kiss - Christine Sixteen Lyrics | AZLyrics.com</title>

<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.4/css/bootstrap.min.css">
<link rel="stylesheet" href="//www.azlyrics.com/bsaz.css">

<!-- HTML5 shim and Respond.js for IE8 support of HTML5 elements and media queries -->
<!--[if lt IE 9]>
<script src="https://oss.maxcdn.com/html5shiv/3.7.2/html5shiv.min.js"></script

In [134]:
def get_raw_az_lyrics(url):
    resp = requests.request("GET",url)
    patt = r'<div>\s*<!-- Usage of azlyrics\.com content by any third-party lyrics provider is prohibited by our licensing agreement\. Sorry about that\. -->\s*([\S\s]*?)\s*<\/div>'
    m = re.search(patt,str(resp.text))
    if not m:
        print(url,"is an edge case >:(")
        return None
    return str(m.group(1))

In [135]:
azlyrics_raw = {}
lim = 10
for i,k in enumerate(az_urls):
    if i > lim:
        break
    azlyrics_raw[k] = get_raw_az_lyrics(az_urls[k])
azlyrics_raw

{('a world without heroes',
  'kiss'): "A world without heroes<br>\r\nIs like a world without sun<br>\r\nYou can't look up to anyone<br>\r\nWithout heroes<br>\r\n<br>\r\nAnd a world without heroes<br>\r\nIs like a never ending race<br>\r\nIs like a time without a place<br>\r\nA pointless thing devoid of grace<br>\r\n<br>\r\nWhere you don't know what you're after<br>\r\nOr if something's after you<br>\r\nAnd you don't know why you don't know<br>\r\nIn a world without heroes<br>\r\n<br>\r\nIn a world without dreams<br>\r\nThings are no more than they seem<br>\r\nAnd a world without heroes<br>\r\nIs like a bird without wings<br>\r\nOr a bell that never rings<br>\r\nJust a sad and useless thing<br>\r\n<br>\r\nWhere you don't know what you're after<br>\r\nOr if something's after you<br>\r\nAnd you don't know why you don't know<br>\r\n<br>\r\nIn a world without heroes<br>\r\nThere's nothing to be<br>\r\nIt's no place for me",
 ('calling dr. love',
  'kiss'): "You need my love baby, oh so bad

In [86]:
print(len(azlyrics))
print(len({(k[0].lower(),k[1].lower()):v for k,v in azlyrics.items() if v[0][2]}))

16207
16204


In [85]:
{(s.lower(),a.lower())for s,a in azlyrics} - set(az_urls.keys())

set()

In [25]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random

ua = UserAgent() # From here we generate a random user agent
proxies = [] # Will contain proxies [ip, port]

# Main function
def main():
  # Retrieve latest proxies
  proxies_req = Request('https://www.sslproxies.org/')
  proxies_req.add_header('User-Agent', ua.random)
  proxies_doc = urlopen(proxies_req).read().decode('utf8')

  soup = BeautifulSoup(proxies_doc, 'html.parser')
  proxies_table = soup.find(id='proxylisttable')

  # Save proxies in the array
  for row in proxies_table.tbody.find_all('tr'):
    proxies.append({
      'ip':   row.find_all('td')[0].string,
      'port': row.find_all('td')[1].string
    })

  # Choose a random proxy
  proxy_index = random_proxy()
  proxy = proxies[proxy_index]

  for n in range(1, 10):
    req = Request('http://icanhazip.com')
    req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')

    # Every 10 requests, generate a new proxy
    if n % 10 == 0:
      proxy_index = random_proxy()
      proxy = proxies[proxy_index]

    # Make the call
    try:
      my_ip = urlopen(req).read().decode('utf8')
      print('#' + str(n) + ': ' + my_ip)
    except: # If error, delete this proxy and find another one
      print(sys.exc_info()[0])
      del proxies[proxy_index]
      print('Proxy ' + proxy['ip'] + ':' + proxy['port'] + ' deleted.')
      proxy_index = random_proxy()
      proxy = proxies[proxy_index]

# Retrieve a random index proxy (we need the index to delete it if not working)
def random_proxy():
  return random.randint(0, len(proxies) - 1)

main()

<class 'KeyboardInterrupt'>
Proxy 212.115.235.12:54608 deleted.
#2: 125.27.251.248

#3: 125.27.251.248

#4: 125.27.251.248

#5: 125.27.251.248

<class 'KeyboardInterrupt'>
Proxy 125.27.251.248:61157 deleted.
#7: 178.162.222.107

#8: 178.162.222.107

#9: 178.162.222.107



In [36]:

def generate_latest_proxies():
    # Retrieve latest proxies
    ua = UserAgent()
    proxies = []
    p_url = 'https://www.us-proxy.org/'
    headers= {'User-Agent':ua.random}
    proxies_doc = requests.get(p_url,headers=headers).text
    #proxies_doc = urlopen(proxies_req).read().decode('utf8')

    soup = BeautifulSoup(proxies_doc, 'html.parser')
    proxies_table = soup.find(id='proxylisttable')
    if not proxies_table:
        sleep(30)
        return []
    # Save proxies in the array
    for row in proxies_table.tbody.find_all('tr'):
        proxies.append({
          'ip':   row.find_all('td')[0].string,
          'port': row.find_all('td')[1].string,
          'https': row.find_all('td')[6].string,
          'anon': row.find_all('td')[4].string
        })
    proxies = [p for p in proxies if p["anon"] in {"elite proxy","anonymous"} and p["https"]=="yes"]
    print("Found {} new proxies".format(len(proxies)))
    return proxies


def make_proxy_req(my_url,prox):    
    http_proxy  = "http://{}:{}".format(prox["ip"],prox["port"])
    https_proxy = "https://{}:{}".format(prox["ip"],prox["port"])

    proxyDict = proxies = {'https': https_proxy}
    try:
        resp = requests.get(my_url,proxies=proxyDict,timeout=5,verify=False)
        return resp.text
    except KeyboardInterrupt as e:
        raise e
    except:
        print(sys.exc_info()[0])
        pass
    return None

In [37]:
proxies = generate_latest_proxies()
for i in range(len(proxies)):
    lyr = make_proxy_req(test_url,proxies[i])
    if lyr:
        print(i)
        proxies = proxies[i:]
        print(proxies[0])
        break
print(lyr)

Found 18 new proxies
<class 'requests.exceptions.ProxyError'>


C:\Users\craut\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


<class 'requests.exceptions.ProxyError'>


C:\Users\craut\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2
{'ip': '184.23.191.119', 'port': '8118', 'https': 'yes', 'anon': 'elite proxy'}
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="width=device-width, initial-scale=1">
<meta name="description" content="Lyrics to &quot;Christine Sixteen&quot; song by Kiss: She's got me dizzy, she sees me through to the end She's got me in her hands and there's no use in..."> 
<meta name="keywords" content="Christine Sixteen lyrics, Kiss Christine Sixteen lyrics, Kiss lyrics">
<meta name="robots" content="noarchive">
<meta property="og:image" content="//www.azlyrics.com/az_logo_tr.png">
<title>Kiss - Christine Sixteen Lyrics | AZLyrics.com</title>

<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.4/css/bootstrap.min.css">
<link rel="stylesheet" href="//www.azlyrics.com/bsaz.css">

<!-- HTML5 shim and Respond.js for IE8 support of HTML5 elements and media queries -->
<!--[if lt IE 

get lyrics from lyrics.az

In [28]:
za_urls = {k:found_songs[k][0] for k in found_songs if found_songs[k][0]}
test_url = list(za_urls.values())[1]
print(test_url)
print(len(za_urls))

https://lyrics.az/kiss/kiss-symphony-alive-iv/forever.html
20605


In [29]:
resp = requests.get(test_url)
print(resp.text)

﻿<!DOCTYPE html>
<html lang="en">
<head>
<title>AZ Lyrics.az | Kiss - Forever lyrics (Kiss Symphony: Alive IV Album)</title>
<script src="/cdn-cgi/apps/head/tAamN4q6hHRKTTIq0yrk24YuMvc.js"></script><link rel="canonical" href="https://lyrics.az/kiss/kiss-symphony-alive-iv/forever.html">
<link rel="amphtml" href="https://app.lyrics.az/kiss/kiss-symphony-alive-iv/forever.html" />
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
<meta name="google-site-verification" content="Myq661bNVy2gMWHtI15LvnTARllJnjAo55E3DAbXu70">
<meta name="apple-itunes-app" content="app-id=988282554">
<link rel="shortcut icon" href="https://www.lyrics.az/favicon.ico">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<style>
.round {border-radius: 100px; border: 1px solid #8f58a9; box-shadow: 0 0 7px #666;}
*{font-family:Verdana,Geneva,Arial,Helvetica,sans-serif;font-size:16px;color:#fff;box-sizing:border-box;margin:0;padding:0}body{background:#35084b}.text-yellow{color:#f0e

In [31]:
def get_raw_za_lyrics(url):
    resp = requests.request("GET",url)
    patt = r'<div class="schemadiv" itemscope itemtype="http://schema\.org/MusicGroup"><p itemprop="description" class="lyric-text" id="lyrics" data-type="textarea" data-pk="1" data-placeholder="Your comments here\.\.\." data-title="Enter comments">\s*([\s\S]*?)\s*</p></div>\s*<a id="correct-lyrics"><u>Correct these Lyrics</u></a>'
    m = re.search(patt,str(resp.text))
    if not m:
        print(url,"is an edge case >:(")
        return None
    return str(m.group(1))
get_raw_za_lyrics(test_url)

'I gotta tell you what I\'m feelin\' inside,<br />\nI could lie to myself, but it\'s true<br />\nThere\'s no denying when I look in your eyes,<br />\nGirl I\'m out of my head over you<br />\nAnd I lived so long believin\' all love is blind<br />\nBut everything about you is tellin\' me this time<br />\n<br />\nIt\'s forever, this time I know and there\'s no doubt in my mind<br />\nForever, until my life is through, girl I\'ll be lovin\' you forever<br />\n<br />\nI hear the echo of a promise I made<br />\nWhen you\'re strong you can stand on your own<br />\nBut those words grow distant as I look at your face<br />\nNo, I don\'t want to go it alone<br />\nI never thought I\'d lay my heart on the line<br />\nBut everything about you is tellin\' me this time<br />\n<br />\n<span style="color:#29013c; font-size:8px;">[Lyrics from: https:/lyrics.az/kiss/kiss-symphony-alive-iv/forever.html]</span><br>\nIt\'s forever, this time I know and there\'s no doubt in my mind<br />\nForever, until my 

In [33]:
lyricsaz_raw = {}
lim = 1000
for i,k in enumerate(za_urls):
    if i > lim:
        break
    lyricsaz_raw[k] = get_raw_za_lyrics(za_urls[k])
len(lyricsaz_raw)

https://lyrics.az/laura-branigan/touch/shattered-glass.html is an edge case >:(
https://lyrics.az/del-shannon/runaway-16-greatest-hits/two-kind-of-teardrops.html is an edge case >:(
https://lyrics.az/whitney-houston/just-whitney/my-love-is-your-love.html is an edge case >:(
https://lyrics.az/kenny-chesney/live-those-songs-again/i-lost-it.html is an edge case >:(
https://lyrics.az/the-game/the-documentary/westside-story.html is an edge case >:(
https://lyrics.az/the-undisputed-truth/law-of-the-land/smiling-faces-sometimes.html is an edge case >:(
https://lyrics.az/brownsville-station/brownsville-station/let-your-yeah-be-yeah.html is an edge case >:(
https://lyrics.az/jefferson-starship/no-protection-performed-as-starship/with-your-love.html is an edge case >:(
https://lyrics.az/jefferson-starship/windows-of-heaven---jeffersons-tree-of-liberty/crazy-feelin.html is an edge case >:(
https://lyrics.az/jefferson-starship/blows-against-the-empire/light-the-sky-on-fire.html is an edge case >:(

ChunkedEncodingError: ('Connection broken: OSError("(10054, \'WSAECONNRESET\')",)', OSError("(10054, 'WSAECONNRESET')",))

In [34]:
len(lyricsaz_raw)

722

In [41]:
lyricsaz_df = pd.DataFrame([[s,a,lyricsaz[(s,a)]] for s,a in lyricsaz if lyricsaz[(s,a)]],columns=["Song","Artist","RawLyrics"])
lyricsaz_df.head()

,Song,Artist,RawLyrics
0,real life,jake owen,Well I grew up in a real town<br />\nWhere the...
1,halle berry (she's fine),hurricane chris featuring supastaar,[Hurricanes words behind the beginning beat]<b...
2,counting the days,joey kid,"Four, three, two, one <br />\n<br />\n(Countin..."
3,work to do,the isley brothers,"I'm taking care of business, baby can't you se..."
4,fever,the mccoys,Never know how much I love you <br />\nNever k...


In [44]:
list_of_words = lambda ly : " ".join(re.split(r"&[\s\S]*?;|<[\s\S]*?>|\[[\s\S]*?\]|[^\w'\\-]|\s",ly.lower())).split()
lists = [list_of_words(ly) for ly in lyricsaz_df["RawLyrics"]]
lyricsaz_df["LyricsList"] = lists
bags = [Counter(lw) for lw in lyricsaz_df["LyricsList"]]
lyricsaz_df["LyricsBag"] = bags
print(lyricsaz_df.shape)
lyricsaz_df.head()

(19864, 5)


,Song,Artist,RawLyrics,LyricsList,LyricsBag
0,real life,jake owen,Well I grew up in a real town<br />\nWhere the...,"[well, i, grew, up, in, a, real, town, where, ...","{'well': 1, 'i': 2, 'grew': 1, 'up': 1, 'in': ..."
1,halle berry (she's fine),hurricane chris featuring supastaar,[Hurricanes words behind the beginning beat]<b...,"[bow, bow, bow, bow, bow, bow, bow, bow, bow, ...","{'bow': 23, 'hurricane': 2, 'bum': 12, 'she': ..."
2,counting the days,joey kid,"Four, three, two, one <br />\n<br />\n(Countin...","[four, three, two, one, counting, the, days, c...","{'four': 1, 'three': 1, 'two': 1, 'one': 1, 'c..."
3,work to do,the isley brothers,"I'm taking care of business, baby can't you se...","[i'm, taking, care, of, business, baby, can't,...","{'i'm': 5, 'taking': 2, 'care': 2, 'of': 3, 'b..."
4,fever,the mccoys,Never know how much I love you <br />\nNever k...,"[never, know, how, much, i, love, you, never, ...","{'never': 2, 'know': 4, 'how': 2, 'much': 2, '..."


In [47]:
pickle.dump(lyricsaz_df,open("lyricsaz_lyricsdata_df.pickle","wb"))

In [61]:
azlyrics_df = pd.DataFrame([[s,a,azlyrics[(s,a)]] for s,a in azlyrics if azlyrics[(s,a)]],columns=["Song","Artist","RawLyrics"])
azlyrics_df.head()

,Song,Artist,RawLyrics
0,neon light,blake shelton,Ever since you left me<br>\nI've been ridin' '...
1,you're so vain,carly simon,Son of a gun.<br>\n<br>\nYou walked into the p...
2,freedom,wham!,Everyday I hear a different story<br>\nPeople ...
3,touch the sky,kanye west featuring lupe fiasco,I gotta testify<br>\nCome up in the spot looki...
4,nothing broken but my heart,celine dion,I've been over you for some time now baby<br>\...


In [62]:
list_of_words = lambda ly : " ".join(re.split(r"&[\s\S]*?;|<[\s\S]*?>|\[[\s\S]*?\]|[^\w'\\-]|\s",ly.lower())).split()
lists = [list_of_words(ly) for ly in azlyrics_df["RawLyrics"]]
azlyrics_df["LyricsList"] = lists
bags = [Counter(lw) for lw in azlyrics_df["LyricsList"]]
azlyrics_df["LyricsBag"] = bags
print(azlyrics_df.shape)
azlyrics_df.head()

(16204, 5)


,Song,Artist,RawLyrics,LyricsList,LyricsBag
0,neon light,blake shelton,Ever since you left me<br>\nI've been ridin' '...,"[ever, since, you, left, me, i've, been, ridin...","{'ever': 1, 'since': 1, 'you': 3, 'left': 1, '..."
1,you're so vain,carly simon,Son of a gun.<br>\n<br>\nYou walked into the p...,"[son, of, a, gun, you, walked, into, the, part...","{'son': 1, 'of': 5, 'a': 5, 'gun': 1, 'you': 4..."
2,freedom,wham!,Everyday I hear a different story<br>\nPeople ...,"[everyday, i, hear, a, different, story, peopl...","{'everyday': 1, 'i': 30, 'hear': 1, 'a': 5, 'd..."
3,touch the sky,kanye west featuring lupe fiasco,I gotta testify<br>\nCome up in the spot looki...,"[i, gotta, testify, come, up, in, the, spot, l...","{'i': 30, 'gotta': 6, 'testify': 8, 'come': 9,..."
4,nothing broken but my heart,celine dion,I've been over you for some time now baby<br>\...,"[i've, been, over, you, for, some, time, now, ...","{'i've': 1, 'been': 6, 'over': 1, 'you': 23, '..."


In [63]:
pickle.dump(azlyrics_df,open("azlyrics_lyricsdata_df.pickle","wb"))

In [50]:
issues = {}
for k in azlyrics:
    if type(azlyrics[k]) != str:
        issues[k] = azlyrics[k]
print(len(issues))

37
